# Agenda
- Text Classification
- Review of text similarity
- POS Tagging Case Study

Data [link](https://drive.google.com/file/d/1V9zmH0y3WXXk689Emn3kDZv7TL35_USP/view?usp=sharing)

In [1]:
import pandas as pd
import numpy as np
path = '/Users/gunnvantsaini/Documents/Data/Kaggle/StackoverflowTags/stack-overflow-data.csv'
data = pd.read_csv(path)

In [2]:
data.head(2)

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net


In [3]:
from sklearn.feature_extraction import text

In [4]:
### Coding question asked on stackoverflow
tfidf=text.TfidfVectorizer(input=data['post'].tolist(),stop_words='english',max_features=4000)
tfidf_matrix=tfidf.fit_transform(data['post'])

In [5]:
tfidf_matrix.shape

(40000, 4000)

In [6]:
X=tfidf_matrix

In [7]:
##### MLP to do document classification ####
len(data['tags'].unique())

20

In [8]:
data['tags'].unique()

array(['c#', 'asp.net', 'objective-c', '.net', 'python', 'angularjs',
       'iphone', 'ruby-on-rails', 'ios', 'c', 'sql', 'java', 'jquery',
       'css', 'c++', 'php', 'android', 'mysql', 'javascript', 'html'],
      dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
enc=LabelEncoder()

In [11]:
y=enc.fit_transform(data['tags'])

In [12]:
y

array([ 5,  3, 15, ...,  4,  2,  1])

In [13]:
enc.classes_

array(['.net', 'android', 'angularjs', 'asp.net', 'c', 'c#', 'c++', 'css',
       'html', 'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby-on-rails', 'sql'],
      dtype=object)

In [14]:
##### Train - test split
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [16]:
import tensorflow as tf

In [17]:
mod=tf.keras.models.Sequential()

Metal device set to: Apple M1


2022-01-14 17:45:50.193378: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-14 17:45:50.193922: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
mod.add(tf.keras.layers.Dense(units=128,activation='relu',input_shape=(4000,)))
mod.add(tf.keras.layers.Dense(units=64,activation='relu'))
mod.add(tf.keras.layers.Dense(units=32,activation='relu'))
mod.add(tf.keras.layers.Dense(units=20,activation='softmax'))

In [19]:
mod.compile(loss="sparse_categorical_crossentropy",metrics=['acc'])

In [20]:
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               512128    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                660       
Total params: 523,124
Trainable params: 523,124
Non-trainable params: 0
_________________________________________________________________


In [21]:
X_train

<32000x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 1197143 stored elements in Compressed Sparse Row format>

In [22]:
mod.fit(X_train.toarray(),y_train,epochs=10,batch_size=32)

2022-01-14 17:46:27.720430: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-14 17:46:27.921899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1669 - acc: 0.6632
Epoch 2/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5786 - acc: 0.8197
Epoch 3/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4930 - acc: 0.8455
Epoch 4/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4424 - acc: 0.8618
Epoch 5/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4020 - acc: 0.8745
Epoch 6/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3692 - acc: 0.8872
Epoch 7/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3368 - acc: 0.8960
Epoch 8/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3065 - acc: 0.9048
Epoch 9/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2776 - acc: 0.9157
Epoch 10/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2517 - acc: 0.9239


In [23]:
text='''I have a Winforms application which uses WCF service (also part of the solution) I am using the Visual Studio 2017 with .Net Framework 4.52 and everything was working fine. I upgraded the project to .Net Framework 4.61 (all projects in current solution including WCF project) after the upgrade I am unable to update service reference in WinForms project and keep getting below error

The current project does not support service references. To enable windows communication foundation support, change the target version of the Microsoft .Net Framework

I trying restarting Visual Studio and my machine but still no luck. I can assure that there is no error in any of the projects because if I downgrade project back to .Net Framework 4.52 then I am able to update service reference successfully so something Visual Studio don't like about 4.61!!

Do any of you have similar error, can someone please provide a way to fix this?'''

In [26]:
vector_doc=tfidf.transform([text])
vec=vector_doc.toarray()
mod.predict(vec).argmax()

0

In [28]:
enc.inverse_transform([0])

array(['.net'], dtype=object)

Data [link](https://drive.google.com/file/d/18a3qXALw6_xEag9SiLsjAsYfn6pMzyrw/view?usp=sharing)

In [29]:
###
path = '/Users/gunnvantsaini/Documents/Work/Jigsaw Academy/Course Review/Textmining_casestudy/products.csv'

In [30]:
data=pd.read_csv(path)

In [31]:
data.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [32]:
data.shape

(49688, 4)

In [33]:
### Mechanism to find similar products ####
### Create tfidf representation ###
from sklearn.feature_extraction import text
corpus=data['product_name'].tolist()

In [37]:
tfidf = text.TfidfVectorizer(input = data['product_name'].tolist())

In [38]:
tfidf_matrix=tfidf.fit_transform(data['product_name'])

In [39]:
tfidf_matrix.shape

(49688, 10737)

In [40]:
#### Compute similarity matrix ####
from sklearn.metrics import pairwise

In [41]:
%%time
sim_matrix=pairwise.cosine_similarity(tfidf_matrix,dense_output=False)

CPU times: user 858 ms, sys: 111 ms, total: 969 ms
Wall time: 1 s


In [42]:
sim_matrix

<49688x49688 sparse matrix of type '<class 'numpy.float64'>'
	with 103527832 stored elements in Compressed Sparse Row format>

In [43]:
import numpy as np

In [44]:
ex=np.array(['a','b','c','d'])

In [45]:
ex[0:2]

array(['a', 'b'], dtype='<U1')

In [46]:
ex[[True,False,True,False]]

array(['a', 'c'], dtype='<U1')

In [48]:
from tqdm import tqdm

In [49]:
results={}
for row_id,row in tqdm(enumerate(sim_matrix),total=sim_matrix.shape[0]):
    target_product=data.iloc[row_id]['product_name']
    target_product_aisle=data.iloc[row_id]['aisle_id']
    idx=row.toarray().argsort()
    rel_idx=idx.reshape(-1)[-6:-1]
    rel_idx=rel_idx[::-1]
    names_sim=data.iloc[rel_idx]['product_name'].values
    sim_aisle_id=data.iloc[rel_idx]['aisle_id'].values
    #### Logic to extract those products whole aisleid is different from target product ###
    logical_idx=(sim_aisle_id!=target_product_aisle)
    names_diff_aisle_ids=names_sim[logical_idx]
    dissimilar_aisle_ids=sim_aisle_id[logical_idx]
    ### Save these values ####
    results[target_product]={'names':names_diff_aisle_ids,
                            'aisle_ids':dissimilar_aisle_ids,
                            'target_aisle_id':target_product_aisle}

100%|████████████████████████████████████| 49688/49688 [00:50<00:00, 992.72it/s]


In [50]:
results['Green Chile Anytime Sauce']

{'names': array(['Green Chile Enchilada Sauce',
        'Green Chile Medium Enchilada Sauce',
        'Medium Green Chile Enchilada Sauce',
        'Mild Enchilada Green Chile Sauce',
        'Mild Green Chile Enchilada Sauce'], dtype=object),
 'aisle_ids': array([30, 30, 30, 30, 30]),
 'target_aisle_id': 5}

In [51]:
target_name=[]
target_aisle=[]
sim_names=[]
sim_aisles=[]
for product in tqdm(results):
    details=results[product]
    for names in details['names']:
        sim_names.append(names)
    for aisle in details['aisle_ids']:
        sim_aisles.append(aisle)
        target_name.append(product)
        ID=details['target_aisle_id']
        target_aisle.append(ID)

100%|█████████████████████████████████| 49688/49688 [00:00<00:00, 484450.15it/s]


In [52]:
pd.DataFrame({'target_name':target_name,'target_aisle':target_aisle,
             'sim_names':sim_names,'sim_aisles':sim_aisles})

,target_name,target_aisle,sim_names,sim_aisles
0,Chocolate Sandwich Cookies,61,Vanilla Sandwich Cookies,37
1,All-Seasons Salt,104,Salsa For All Seasons Tropical Mango,51
2,All-Seasons Salt,104,Salsa For All Seasons Medium Chipotle Salsa,67
3,All-Seasons Salt,104,Salsa For All Seasons Half Guacamole Half Roas...,67
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,Smart Ones Classic Favorites Thin Crust Pepper...,79
...,...,...,...,...
101232,En Croute Roast Hazelnut Cranberry,42,"Granola Bar, Fig, Cranberry & Hazelnut",3
101233,En Croute Roast Hazelnut Cranberry,42,Hencho en Mexico,77
101234,En Croute Roast Hazelnut Cranberry,42,Hazelnut Light Roast K-Cup Packs,26
101235,En Croute Roast Hazelnut Cranberry,42,Organic Cranberry,98
